In [ ]:
import pandas as pd
import pyodbc
import os

folder_path = r"C:\Users\Avi\OneDrive - Kasmo Digital Private Limited\Desktop\Python_ETL"

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-N8TVR49;'
    'DATABASE=python_ETL;'
    'UID=sa;'
    'PWD=sql123'
)
cursor = conn.cursor()

for file_name in os.listdir(folder_path):
    if file_name.endswith(('.csv', '.xlsx')):
        file_path = os.path.join(folder_path, file_name)
        table_name = os.path.splitext(file_name)[0]

        print(f"\n Processing `{file_name}` into `{table_name}`...")

        try:
            if file_name.endswith('.csv'):
                df = pd.read_csv(file_path, encoding='utf-8')
            else:
                df = pd.read_excel(file_path, engine='openpyxl')
        except Exception as e:
            print(f"Failed to read {file_name}: {e}")
            continue

        df.columns = [str(col).strip().replace(" ", "_").replace("-", "_") for col in df.columns]
        df.fillna('', inplace=True)  

        create_sql = f"CREATE TABLE [{table_name}] ("
        create_sql += ', '.join(f"[{col}] NVARCHAR(MAX)" for col in df.columns) + ")"

        try:
            cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE [{table_name}]")
            cursor.execute(create_sql)
            conn.commit()
            print(f"Table `{table_name}` created.")
        except Exception as e:
            print(f"Error creating table `{table_name}`: {e}")
            continue

        insert_sql = f"INSERT INTO [{table_name}] ({', '.join(f'[{col}]' for col in df.columns)}) VALUES ({', '.join('?' for _ in df.columns)})"
        inserted_count = 0
        skipped_count = 0

        for _, row in df.iterrows():
            try:
                cursor.execute(insert_sql, tuple(str(val) for val in row))
                inserted_count += 1
            except Exception as e:
                print(f"Failed to insert row: {e}")
                skipped_count += 1

        conn.commit()
        print(f"Inserted {inserted_count} rows into: {table_name}")
        if skipped_count > 0:
            print(f"kipped {skipped_count} rows due to errors.")

cursor.close()
conn.close()




📥 Processing `order_data_1.csv` into `order_data_1`...
✅ Table `order_data_1` created.
✅ Inserted 2000 rows into: order_data_1

📥 Processing `transaction_data.csv` into `transaction_data`...
✅ Table `transaction_data` created.


C:\Users\Avi\AppData\Local\Temp\ipykernel_9672\729625898.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)  # Replace NaN with empty string


✅ Inserted 1000 rows into: transaction_data

📥 Processing `us_customer_data_1.csv` into `us_customer_data_1`...
✅ Table `us_customer_data_1` created.
✅ Inserted 1000 rows into: us_customer_data_1

🎉 All files processed successfully.


In [7]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-N8TVR49;'
    'DATABASE=python_ETL;'
    'UID=sa;'
    'PWD=sql123'
)

In [12]:
cursor = conn.cursor()
cursor.execute("select top 5 * from order_data_1")
for row in cursor.fetchall():
    print(row)


('892a07a4-d252-4775-85e0-73077143e1c6', '966', '2024-11-27', '317.64', 'Cancelled', 'Home & Garden')
('ae160758-e187-47b2-9350-032f88f55491', '345', '2023-03-27', '645.87', 'Completed', 'Home & Garden')
('7c50456e-6123-45cc-aa19-128bef3754d6', '503', '2024-03-31', '880.86', 'Pending', 'Clothing')
('c788b56b-3716-4cd9-a827-d4dc401ba00c', '385', '2023-08-09', '876.83', 'Cancelled', 'Home & Garden')
('925ab5b1-adb1-4302-a70b-1c2db724e02b', '817', '2023-04-19', '264.53', 'Pending', 'Home & Garden')


In [11]:
cursor.execute("select top 5 * from transaction_data")
for row in cursor.fetchall():
    print(row)

('1', '565', '2992.47', '2025-03-10 01:20:54', 'Sports', 'Debit Card', 'New York')
('2', '323', '2041.87', '2025-01-02 15:24:19', 'Clothing', 'Cash', 'New York')
('3', '398', '107.35', '2025-02-16 03:49:01', 'Beauty', 'Debit Card', 'Online')
('4', '19', '', '2025-04-30 15:26:23', 'Sports', 'Debit Card', 'Los Angeles')
('5', '547', '3063.28', '2025-06-14 04:28:53', 'Clothing', 'PayPal', 'Los Angeles')


In [13]:
cursor.execute("select top 5 * from us_customer_data_1")
for row in cursor.fetchall():
    print(row)

('1', 'Michelle Kidd', 'vayala@example.net', '619-723-4258', 'USNS Santiago, FPO AE 80872', '25-01-2025', 'Gold')
('2', 'Brad Newton', 'taylorcatherine@example.net', '537-674-1158', '38783 Oliver Street, West Kristenborough, MT 99752', '13-07-2023', 'Silver')
('3', 'Larry Torres', 'dsanchez@example.net', '810-256-4505', '6845 Steele Turnpike, West Erikabury, UT 37487', '18-08-2023', 'Bronze')
('4', 'Kimberly Price', 'jessicaknight@example.com', '423-222-9779', '1631 Alexis Meadows, Lake Amanda, CA 75179', '08-12-2024', 'Gold')
('5', 'Matthew Phillips', 'qwilliams@example.com', '220-763-3522', '2274 Williams Heights Suite 895, Andersonhaven, OR 80565', '03-02-2024', 'Gold')


In [15]:

new_order = (
    'test-0001-uuid-9876',  
    '999',                   
    '2025-06-19',            
    699.99,                  
    'Shipped',              
    'Books'                  
)

insert_query = """
INSERT INTO order_data_1 (
    order_id, customer_id, order_date, order_amount, order_status, product_category
) VALUES (?, ?, ?, ?, ?, ?)
"""


cursor.execute(insert_query, new_order)
conn.commit()

print("New row inserted into `order_data_1` successfully.")


cursor.close()
conn.close()


New row inserted into `order_data_1` successfully.


In [16]:
import re

In [23]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-N8TVR49;'
    'DATABASE=python_ETL;'
    'UID=sa;'
    'PWD=sql123'
)

In [27]:
cursor = conn.cursor()

In [34]:
prefux = r'^(Mr\.|Mrs\.|Miss|Ms\.|Dr\.)\s*'
suffix = r'\s*(Jr\.|Sr\.|II|III|MD|DDS|PhD)$'

In [35]:
cursor.execute("select customer_id, name from us_customer_data_1")
rows = cursor.fetchall()

In [36]:
for row in rows:
    customer_id,name = row
    newname = re.sub(prefux, '', name)
    newname = re.sub(suffix, '', newname)
    newname = newname.strip()
    update_query = "UPDATE us_customer_data_1 SET name = ? WHERE customer_id = ?"
    cursor.execute(update_query, (newname, customer_id))
    conn.commit()

In [38]:
df.loc[993]

customer_id                                                  994
name                                               Mario Odom MD
email                                  sheilarobbins@example.org
phone                                               773-566-3560
address              87583 Tabitha Bypass, West Denise, LA 20244
registration_date                                     24-03-2025
loyalty_status                                            Silver
Name: 993, dtype: object

In [39]:
try:
    cursor.execute("alter table us_customer_data_1 add first_name Nvarchar(200),last_name Nvarchar(200)")
    conn.commit()
except:
    pass

In [42]:
cursor.execute("select customer_id,name from us_customer_data_1")
rows = cursor.fetchall()

In [46]:
for customer_id,full_name in rows:
    name_parts = full_name.strip().split()

    if len(name_parts) >=2:
        first_name = name_parts[0]
        last_name = ' '.join(name_parts[1:])
    elif len(name_parts) == 1:
        first_name = name_parts[0]
        last_name = ''
    else:
        first_name = ''
        last_name = ''  

    update_query = "UPDATE us_customer_data_1 SET first_name = ?, last_name = ? WHERE customer_id = ?"
    cursor.execute(update_query,(first_name,last_name,customer_id))

conn.commit()




In [47]:
cursor.close()
conn.close()

In [45]:
df.loc[1]

customer_id                                                          2
name                                                       Brad Newton
email                                      taylorcatherine@example.net
phone                                                     537-674-1158
address              38783 Oliver Street, West Kristenborough, MT 9...
registration_date                                           13-07-2023
loyalty_status                                                  Silver
Name: 1, dtype: object

In [48]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-N8TVR49;'
    'DATABASE=python_ETL;'
    'UID=sa;'
    'PWD=sql123'
)

In [49]:
cursor = conn.cursor()

In [55]:
cursor.execute("""CREATE VIEW test_view AS
SELECT 
    c.customer_id AS customer_id_c,
    c.name,
    c.email,
    c.phone,
    c.address,
    c.registration_date,
    c.loyalty_status,
    o.order_id,
    o.order_date,
    o.order_amount,
    o.order_status,
    o.product_category AS order_product_category,
    t.transaction_id,
    t.amount AS transaction_amount,
    t.transaction_date,
    t.product_category AS transaction_product_category,
    t.payment_method,
    t.store_location

FROM 
    us_customer_data_1 AS c
JOIN 
    order_data_1 AS o ON c.customer_id = o.customer_id
JOIN 
    transaction_data AS t ON c.customer_id = t.customer_id;
""")
conn.commit()

In [67]:
try:
    cursor.execute("alter table us_customer_data_1 add loyalty_Score Nvarchar(200)")
    conn.commit()
except:
    pass

In [68]:
conn.commit()

In [69]:
conn.close()

In [70]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-N8TVR49;'
    'DATABASE=python_ETL;'
    'UID=sa;'
    'PWD=sql123'
)

In [71]:
cursor = conn.cursor()

In [72]:
cursor.execute("update us_customer_data_1 set loyalty_Score = Case when loyalty_status = 'Gold' then '0' when loyalty_status = 'Silver' then '1' else '2' end")
conn.commit()

In [73]:
conn.close()

In [74]:
df.isnull().sum()

customer_id          0
name                 0
email                0
phone                0
address              0
registration_date    0
loyalty_status       0
dtype: int64